In [1]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 企业版, 17134, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]'

'2019-01-01 23:02:52'

- **@author**: run_walker
- **@references**:
    1. [原码, 反码, 补码 详解](https://www.cnblogs.com/zhangziqiu/archive/2011/03/30/ComputerCode.html)
    2. [原码 反码 补码 及 python的位运算](https://blog.csdn.net/u013061183/article/details/78525807)
    3. [Python小技巧：负数的补码表示](https://www.jianshu.com/p/96ea0b077051)
    4. [你不知道的按位运算](http://python.jobbole.com/85168/)
    5. [为什么8位有符号类型的数值范围是-128~127](https://www.jianshu.com/p/0ad27ad30a43)
    6. [位操作基础篇之位操作全面总结](https://blog.csdn.net/morewindows/article/details/7354571)
    7. [Python 运算符](http://www.runoob.com/python/python-operators.html)

注意：
1. 尽管$-2^{31}$在原码和反码中无法表示，但是补码却可以表示为$\overline{10\cdots0}$，由$-2^{31}+1$的补码$\overline{10\cdots01}$减去$1$得到。
2. 按照正整数和负整数两种规则，$0$有两种原码和反码，但是补码只有一种，为$\overline{0\cdots0}$

因为计算机中数值以补码形式储存和运算，所以32位有符号整数的范围为$[-2^{31},~2^{31}-1]$。

# 补码

## 定义
1. 正整数的符号位为0，反码和补码仍为原码
2. 负数的符号位为1，反码为原码除符号位外取反（1取0，0取1），补码为其反码加1。也即对于$k>0$，有$(-k)_{\mbox{补}}=\sim k_{\mbox{原}}+1$。

## 性质
1. 奇数的补码最后一位为1，偶数的补码最后一位为0。
2. 对于整数$k$，记其补码为$\overline{x_0x_1\cdots x_{31}}$，则有$k=\overline{0x_1\cdots x_{31}}-x_0*2^{31}$
3. 求补码的运算迭代两次后，得到的结果为原码。

Proof 2:
1. 若$k$为非负整数，该结论是显然的。
2. 当$k$为负整数时，$x_1$到$x_{31}$中至少存在一个1，记其反码为$\overline{1y_1\cdots y_{31}}$，则有
$$\overline{0x_1\cdots x_{31}}-1=\overline{0y_1\cdots y_{31}}$$
记其原码为$\overline{1z_1\cdots z_{31}}$，则有
$$\overline{0y_1\cdots y_{31}}+\overline{0z_1\cdots z_{31}}=\overline{1\cdots1}=2^{31}-1$$
则有
$$k=-\overline{0z_1\cdots z_{31}}=\overline{0x_1\cdots x_{31}}-2^{31}$$

Proof 3:

1. 对于非负整数，显然。
2. 对于负整数$-k$，记其原码为$\overline{1x_1\cdots x_{31}}$，补码为$\overline{1y_1\cdots y_{31}}$，设$a_i=1-x_i,~b_i=1-y_i,~\forall 1\leq i\leq 31$。则由补码的定义2可知
$$\overline{1y_1\cdots y_{31}}=\sim \overline{0x_1\cdots x_{31}}+1=\overline{1a_1\cdots a_{31}}+1$$
则有
$$\overline{0y_1\cdots y_{31}}=\overline{0a_1\cdots 0_{31}}+1$$
两边乘以$-1$后同时加上$\overline{1\cdots1}$可得
$$\overline{1b_1\cdots b_{31}}=\overline{1x_1\cdots x_{31}}-1$$
对其补码再做补码运算可以得到$\sim \overline{0y_1\cdots y_{31}}+1$，同样地有，
$$
\sim \overline{0y_1\cdots y_{31}}+1=\overline{1b_1\cdots b_{31}} + 1=\overline{1x_1\cdots x_{31}}
$$


## 转换
对于负整数$-k$，记其原码为$\overline{1x_1\cdots x_{31}}$，补码为$\overline{1y_1\cdots y_{31}}$，总结上述的定义和性质，可以得到他们它们间的转换如下：

1. 数值和原码之间。将正整数$k$的二进制表示的符号位置为1则为负整数$-k$的原码，$k=\overline{0x_1\cdots x_{31}}$。
2. 原码和补码之间。由定义2和性质3可知：补码$\overline{1y_1\cdots y_{31}}=\sim\overline{0x_1\cdots x_{31}}+1$，原码$\overline{1x_1\cdots x_{31}}=\sim\overline{0y_1\cdots y_{31}}+1$。
3. 补码和数值之间。由性质2可知：$-k=\overline{0y_1\cdots y_{31}}-2^{31}$。

In [32]:
def intToBin32(num):
    """
    10进制有符号整数 -> 32位补码
    """
    return (bin(0xFFFFFFFF & num)[2:]).zfill(32)


def bin32ToInt(s):
    """
    32位补码 -> 10进制有符号整数
    """
    return int(s[1:], 2) - int(s[0]) * (1 << 31)

In [31]:
intToBin32(0)
intToBin32(1)
intToBin32(-1)
intToBin32(127)
intToBin32(-127)

'00000000000000000000000000000000'

'00000000000000000000000000000001'

'11111111111111111111111111111111'

'00000000000000000000000001111111'

'11111111111111111111111110000001'

In [39]:
bin32ToInt(intToBin32(0))
bin32ToInt(intToBin32(1))
bin32ToInt(intToBin32(-1))
bin32ToInt(intToBin32(127))
bin32ToInt(intToBin32(-127))

0

1

-1

127

-127

## 陷阱
python中int类型通常占32个二进制位，但事实上是不限长度的（即不存在高位溢出）。将负数的32位补码在python中用int转回10进制得到的不是原负数，而是真值（正常转换的正数）。 

In [41]:
intToBin32(-1)
int(intToBin32(-1), 2)

'11111111111111111111111111111111'

4294967295

# 位运算符

注意：
1. 位运算是对对象（只能是整型）的补码按位进行操作，得到的仍是补码，转换回原码后才能得知最终结果。
2. 除$\sim{}$外，其他位运算符优先级均低于加减法，所以要注意加括号。

## 运算符优先级
|运算符|描述|
|:--|:--|
|**|指数 (最高优先级)|
|~ + -|按位翻转，正，负|
|* / % //|乘，除，取模和整除|
|+ -|加法减法|
|>> <<|右移，左移|
|&|按位与|
|$|$ ^|按位或，异或|
|<=  <  > >=|比较运算符|
|<> == !=|等于运算符|
|=, %=, /=, //=, -=, +=, *= ,**=|赋值运算符|
|is, is not|身份运算符|
|in, not in|成员运算符|
|not, or, and|逻辑运算符|

## $\&$ 与

### 性质
1. $x~\&~ x=x,~x~\&~0=x~\&~(\sim x)=0,~-1~\&~x=x$
2. 对于正整数$k$，$k$为2的幂次方，当且仅当$k~\&~(k-1)=0$
3. $1~\&~ 2k=0,~1~\&~(2k+1)=1$，特别地，若$x\in\{0,~1\}$，则有$1~\&~x=x$
4. $x~\&~(2^k-1)=x~\%~2^k$，$k$取1的时候也即性质3

Proof 1:

不管是多少位，$-1$的每个数位都是1。

Proof 4:

记$x$的补码为$\overline{x_{32}\cdots x_{k+1}x_k\cdots x_1}$，则有
$$x~\&~(2^k-1)=\overline{0\cdots0x_k\cdots x_1}$$
由补码的性质2可知
$$x~\%~2^k=(\overline{0x_{31}\cdots x_1}-x_{32}*2^{31})~\%~2^k=\overline{0x_{31}\cdots x_1}~\%~2^k=\overline{0\cdots0 x_k\cdots x_1}$$

### 应用

#### 判断奇偶
根据性质3

In [4]:
def is_odd(num):
    return bool(num & 1)

In [5]:
is_odd(-5)
is_odd(-4)
is_odd(0)
is_odd(4)
is_odd(5)

True

False

False

False

True

#### 模$2^k$求余
根据性质4

In [6]:
def mod_2_pow_k(num, k):
    return num & ((1 << k) - 1) 

In [7]:
for k in range(5):
    for num in range(-100, 100):
        if num % (1 << k) != mod_2_pow_k(num, k):
            print(num, k)

## $|$ 或

### 性质
1. $x~|~x=x~|~0=x,~x~|~(\sim x)=-1,~x~|~-1=-1$

In [3]:
for i in range(-100, 100):
    if i | (~i) != -1:
        print(i)
    if i & (~i) != 0:
        print(i)
    if i | -1 != -1:
        print(i)

## $\hat{}$ 异或
当bool类型数据和整数进行异或运算时，True视为1，False视为0。

In [30]:
1 ^ (-9)
1 ^ 9
1 ^ -8
1 ^ 8

-10

8

-7

9

### 性质
1. $x~\hat{}~0=x,~x~\hat{}~x=0$。特别地，若$x\in\{0,~1\}$，则有$1~\hat{}~x=1-x$。
2. 交换律：$x~\hat{}~y=y~\hat{}~x$
3. 结合律：$(x~\hat{}~y)~\hat{}~z=x~\hat{}~(y~\hat{}~z)$
4. 若有$a~\hat{}~b=c$，则有$a~\hat{}~c=b,~b~\hat{}~c=a$
5. $1~\hat{}~(2k+1)=2k,~1~\hat{}~2k=2k+1,~2k~\hat{}~(2k+1)=1$。
6. $-1~\hat{}~k=\sim k=-(k+1)$
7. 对于非负整数$x$和$y$，若$\exists n,~st.~x+y=2^n-1=\overline{1\cdots1}$，其中共$n$位1，则$x~\hat{}~y=2^n-1$。

<div class="alert alert-block alert-warning">
    <i class="fa fa-sticky-note" aria-hidden="true"><b> Note:</b></i>
    异或不具有<b>分配率</b>。
</div>

**Proof 3:**

易证$\forall i,j,k\in\{0,1\},~(i\hat{}j)\hat{}k=i\hat{}(j\hat{}k)$。$\forall x,y,z$，记$x=\overline{x_1x_2\cdots x_n},~y=\overline{y_1y_2\cdots y_n},~z=\overline{z_1z_2\cdots z_n}$，其中$x_i,y_i,z_i\in\{0,1\},~\forall i\in [1,~n]$。

**Proof 4:**

$a~\hat{}~c=a~\hat{}(a~\hat{}~b)=(a~\hat{}~a)~\hat{}~b=0~\hat{}~b=b$

**Proof 5:**

$2k$的最后一位为$0$，$2k+1$的最后一位为$1$，其他位置全部一致，所以$2k~\hat{}~(2k+1)=1$

**Proof 6:**

$-1$的补码为$\overline{1\cdots 1}$，32位均为1，则$-1~\hat{}~k$也即相当于对$k$取反$\sim k$，由取反的性质1可知$-1~\hat{}~k=-(k+1)$。

### 应用

#### 交换两数

In [5]:
# 通过异或
a, b = 5, 20
a, b 
a = a ^ b 
b = b ^ a 
a = a ^ b 
a, b 

(5, 20)

(20, 5)

In [7]:
# 通过加法
a, b = 5, 20
a, b 
a = a + b
b = a - b
a = a - b 
a, b 

(5, 20)

(20, 5)

In [6]:
# python特有语法
a, b = 5, 20
a, b 
a, b = b, a  # 使用了额外的空间
a, b 

(5, 20)

(20, 5)

#### 求绝对值
由异或性质5及右移运算性质可知。

In [25]:
def bit_abs(num):
    sign = num >> 31
    return (num ^ sign) - sign

In [51]:
for i in range(-100, 100):
    if bit_abs(i) != abs(i):
        print(i)

## $\sim$ 取反

### 性质
1. $\sim k=-(k+1)$
2. $\sim\sim k=k$

Proof 1:

1. $k>0$。则$k$的补码中必然存在至少一个数位为1，记$k=\overline{00\cdots01x_1x_2\cdots x_n}$，其中1左侧有若干个0，当1右侧没有数位或者全为1时结论易证，所以不妨设$x_1$到$x_n$中至少存在一个0，记$\sim k=\overline{11\cdots10y_1y_2\cdots y_n}$，则有
$$\overline{x_1\cdots x_n} + \overline{y_1\cdots y_n} = \overline{11\cdots1}$$
且$y_1$到$y_n$中至少存在一个1，则$\overline{y_1\cdots y_n}>1$，从符号位可知$\sim k$是负数，记$\sim k$的反码为$\overline{11\cdots10a_1a_2\cdots a_n}$，则有
$$\overline{y_1\cdots y_n}-1=\overline{a_1\cdots a_n}$$
记$\sim k$的原码为$\overline{10\cdots01b_1b_2\cdots b_n}$，则有
$$\overline{a_1\cdots a_n} + \overline{b_1\cdots b_n}=\overline{1\cdots1}$$
根据以上三个等式可知
$$\overline{b_1\cdots b_n}=\overline{x_1\cdots x_n} + 1$$
根据正整数$k$和负整数$\sim k$的原码及等式(4)可知
$$\sim k=-(k+1)$$
2. $k=0$，易证。
3. $k<0$，类似于1的证明。

Proof 2:

根据性质1，$~\sim\sim k=\sim( -(k+1))=-(-(k+1)+1)=k$

### 应用
也即$\sim k+1=-k$，可用于求$k$的相反数。这同时也说明了$-k$的补码为$k$的原码取反后加1，这和负数补码的规则是一致的。

## $\gg$ 右移

### 定义
将补码按位向右移动，左边缺少的地方补符号位（即正数补0，负数补1），相当于除以2的次方后取整（注意性质2）。 

### 性质
1. $-1\gg n=-1$
2. 对于正数，不断向右移位最终得到0，而负数最终得到-1

# 一些等式

## $a+b=(a~\hat{}~b)+[(a~\&~b)\ll 1]$

Proof:

易证$\forall a,b\in\{0,~1\}$满足该等式，而位操作符都是按位进行的，证毕。

## $a~\hat{}~b=(\sim{a})~\hat{}~(\sim{b})$

## $a~|~b=(a~\&~b)+(a~\hat{}~b)$

Proof:

易证$\forall a,b\in\{0,~1\}$满足该等式，而位操作符都是按位进行的，证毕。

## $a=(a~\&~b)+(a~\&~(\sim{b}))$

Proof:

易证$\forall a,b\in\{0,~1\}$满足该等式，而位操作符都是按位进行的，证毕。

In [2]:
for a in range(-100, 100):
    for b in range(-1000, 1000):
        x1 = a & b 
        x2 = a & (~b)
        if a != x1 + x2:
            print(a, x1, x2)

# 趣味应用

## 取出特定位置
对于$x\in\{0,1\}$，另一个操作数分别为0、1、-1、$x$时，三种逻辑运算的结果如下表：

|操作数| $\&$ 与|$|$ 或|$\hat{}$ 异或|
|:--|:--|:--|:--|
|0|0|$x$|$x$|
|1|$x$|1|$1-x$|
|-1|$x$|-1|$\sim{x}$|
|$x$|$x$|$x$|0|

设整数$k$的补码为$\overline{x_1x_2\cdots x_{32}}$。

1. 如果想每隔2位取出首位，第二位以0填充，可以进行运算$\overline{x_10x_30\cdots x_{31}0}=k~\&~\overline{1010\cdots1010}=k~\&~\mbox{0xAAAAAAAA}$
2. 类似地，$0x_20x_4\cdots 0x_{32}=k~\&~\mbox{0x55555555}$

## 找到右侧第一个1
若整数$x$的补码从右向左数第一个1在第k位，则有$x~\&~-x=\overline{0\cdots010\cdots0}$，其中等式右侧的1在从右向左数第$k$位上。

Proof:

1. 对于正整数$x$，记其补码为$\overline{0x_1\cdots x_k10\cdots0}$，从左向右数第0位符号位为0，第$k+1$位为1，右侧没有数位或者均为0，记$-x$的补码为$\overline{1y_1\cdots y_k01\cdots1}$，其中$y_i=\sim{x_i},~\forall 1\leq i\leq k$，则$x~\&~-x=\overline{0\cdots010\cdots0}$
2. 对于负整数$-x$同理。

## 高低位交换
将一个32位的有符号整数的前16位和后16位交换。

In [42]:
def high_low_swap(num):
    return ((num >> 16) & 0xFFFF) | (num << 16)

In [43]:
for i in range(-100, 100):
    s1 = intToBin32(i)
    s2 = intToBin32(high_low_swap(i))
    if s1 != s2[16:] + s2[:16]:
        print(i)

## 二进制逆序
[190. 颠倒二进制位](https://leetcode-cn.com/problems/reverse-bits/)

## 二进制中1的个数
[191. 位1的个数](https://leetcode-cn.com/problems/number-of-1-bits/)

## 格雷编码
详见 [89. Gray Code.ipynb](../LeetCode/89. Gray Code.ipynb)

## 只出现一次的数字
详见 [136 & 137 & 260 & 287. 只出现一次的数字](../LeetCode/136 & 137 & 260 & 287. 只出现一次的数字.ipynb)

## 空间压缩
[LeetCode 661. 图片平滑器](https://leetcode-cn.com/problems/image-smoother/)

已知原数值和修改后的值都在$0$到$255$之间，所以可以用二进制表示的后$8$位记录原数值，中$8$位记录修改后的值，使用时通过位运算取出。